In [1]:
import sys
sys.path.append('..')

from src.pipeline import *

import sqlite3

conn = sqlite3.connect(DB_NAME)
cursor = conn.cursor()

# Convert timestamp from JST to UTC (-09:00) for all records
cursor.execute("UPDATE metadata SET created_at = datetime(created_at, '-9 hours')")
conn.commit()


c:\Users\hiroga\miniconda3\envs\pokemon-pal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# insert nobg job to SQLite manually.
from datetime import datetime
from src.pipeline import Step, create_job

# Insert a nobg job manually
create_job(Step.nobg, datetime.utcnow())
